In [ ]:
from ultralytics import YOLO
import cv2
import time
from IPython.display import display, clear_output
import PIL.Image

# Load YOLOv8 nano model
model = YOLO('yolov8n.pt')

# Open video file
cap = cv2.VideoCapture(r"C:\Users\mithi\Downloads\testing\breaking video3.mp4")

# Parameters for braking detection
prev_area = None
prev_time = None
brake_threshold = 1.5
frame_skip = 2
frame_count = 0
scale = 0.6

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video ended or cannot be read.")
        break

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue

    orig_h, orig_w = frame.shape[:2]
    resized = cv2.resize(frame, (int(orig_w * scale), int(orig_h * scale)))
    results = model(resized, imgsz=480, stream=True)

    sudden_brake_detected = False

    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            label = model.names[cls_id]

            if label in ['car', 'truck', 'bus', 'motorbike']:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                w, h = x2 - x1, y2 - y1
                area = w * h

                # Scale coords back to original size
                x1 = int(x1 / scale)
                y1 = int(y1 / scale)
                x2 = int(x2 / scale)
                y2 = int(y2 / scale)

                # Draw detection box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{label}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 3)

                now = time.time()
                if prev_area is not None and prev_time is not None:
                    dt = now - prev_time
                    if dt > 0:
                        growth_rate = (area - prev_area) / prev_area
                        if growth_rate > brake_threshold:
                            sudden_brake_detected = True

                prev_area = area
                prev_time = now

    if sudden_brake_detected:
        cv2.putText(frame, "⚠ SUDDEN BRAKING AHEAD!", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

    # Convert BGR (OpenCV) to RGB (PIL)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = PIL.Image.fromarray(frame_rgb)

    # Clear previous output and display the frame
    clear_output(wait=True)
    display(img_pil)

cap.release()
